# EDA precipitación de Azure Table Storage

### Librerías

In [48]:
import pandas as pd
import os
import plotly.express as px
import dash

from azure.data.tables import TableClient
from dotenv import load_dotenv, find_dotenv
from dash import dcc, html
from dash.dependencies import Input, Output


In [49]:
load_dotenv(find_dotenv())

True

In [50]:
connection_string = os.getenv("AZ_CONNECTION_STRING")

### Funciones necesarias

In [51]:

# Crear conexión de una tabla específica dentro del servicio de Azure Table Storage
def set_table_service(connection_string, table):
    """Crear servicio de conexión a Azure Table Storage"""
    return TableClient.from_connection_string(
        conn_str=connection_string, table_name=table
    )


# Obtener datos de Table Storage
def get_data_from_table_storage_table(table_service, filter_query):
    """Recuperar datos de Table Storage"""
    for record in table_service.query_entities(filter_query):
        yield record


# Crear DataFrame con los datos de la tabla consultada
def get_dataframe_from_table_storage_table(table_service, filter_query):
    """Crear un DataFrame con la data del Table Storage"""
    return pd.DataFrame(get_data_from_table_storage_table(table_service, filter_query))

### Creando DataFrame con Data de Az Table Storage

In [52]:
table_name_precipitacion = "PRECIPITACIONCurated"
filterQuery = "PartitionKey ne 'random'"

In [53]:
table_service_precipitacion = set_table_service(connection_string, table_name_precipitacion)
df_precipitacion = get_dataframe_from_table_storage_table(table_service_precipitacion, filterQuery)
print(f"Shape: {df_precipitacion.shape}")
print(f"dtypes: {df_precipitacion.dtypes}")
df_precipitacion.head()

Shape: (261362, 15)
dtypes: PartitionKey            object
RowKey                  object
TimeStamp                int64
codigoestacion          object
codigosensor            object
date                    object
departamento            object
descripcionsensor       object
latitud                 object
longitud                object
municipio               object
nombreestacion          object
precipitacion_total    float64
unidadmedida            object
zonahidrografica        object
dtype: object


,PartitionKey,RowKey,TimeStamp,codigoestacion,codigosensor,date,departamento,descripcionsensor,latitud,longitud,municipio,nombreestacion,precipitacion_total,unidadmedida,zonahidrografica
0,DefaultPartitionKey,00003699-9864-405f-83fb-afd64270a798,20231026,0042077020,0240,2021-07-14T00:00:00.000,VAUPÉS,Precipitacion,1.26,-70.239,MITÚ,MITU,10.0,mm,VAUPES
1,DefaultPartitionKey,0000400d-4c13-40cd-88de-51c6d305d1ef,20231026,0012015110,0240,2021-12-12T00:00:00.000,ANTIOQUIA,Precipitacion,7.671138889,-76.69405556,CHIGORODÓ,CHIGORODO - AUT,0.1,mm,CARIBE - LITORAL
2,DefaultPartitionKey,00007b0b-ec72-4c5e-8a80-aaf73c645a67,20231026,3526500201,0240,2023-01-17T00:00:00.000,VICHADA,Precipitacion,5.48088889,-70.42130556,LA PRIMAVERA,LA PRIMAVERA,0.0,mm,ORINOCO
3,DefaultPartitionKey,0000ca4d-4c22-41a8-a599-d20fc79fd83a,20231026,0024025030,0240,2021-11-15T00:00:00.000,BOYACÁ,Precipitacion,5.966388889,-73.16389139,PAIPA,LA SIERRA - AUT,5.0,mm,SOGAMOSO
4,DefaultPartitionKey,0000d2d6-8c7b-4120-9078-177e8855eef2,20231026,0035025110,0240,2023-01-02T00:00:00.000,META,Precipitacion,4.057361111,-73.46791667,VILLAVICENCIO,LA LIBERTAD - AUT,0.5,mm,META


In [54]:
df_precipitacion = df_precipitacion.drop(['PartitionKey', 'RowKey','TimeStamp'], axis=1)
df_precipitacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261362 entries, 0 to 261361
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   codigoestacion       261362 non-null  object 
 1   codigosensor         261362 non-null  object 
 2   date                 261362 non-null  object 
 3   departamento         261362 non-null  object 
 4   descripcionsensor    261362 non-null  object 
 5   latitud              261362 non-null  object 
 6   longitud             261362 non-null  object 
 7   municipio            261362 non-null  object 
 8   nombreestacion       261362 non-null  object 
 9   precipitacion_total  261362 non-null  float64
 10  unidadmedida         261362 non-null  object 
 11  zonahidrografica     261362 non-null  object 
dtypes: float64(1), object(11)
memory usage: 23.9+ MB


In [55]:
#df_precipitacion = df_precipitacion.drop_duplicates(subset=['codigoestacion','date','municipio','departamento'])
df_precipitacion = df_precipitacion.copy()

df_precipitacion['date'] = pd.to_datetime(df_precipitacion['date'], errors='coerce')
df_precipitacion['date'] = df_precipitacion['date'].dt.strftime('%Y-%m-%d')
df_precipitacion['date'] = pd.to_datetime(df_precipitacion['date'])

df_precipitacion.info()
df_precipitacion.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261362 entries, 0 to 261361
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   codigoestacion       261362 non-null  object        
 1   codigosensor         261362 non-null  object        
 2   date                 261362 non-null  datetime64[ns]
 3   departamento         261362 non-null  object        
 4   descripcionsensor    261362 non-null  object        
 5   latitud              261362 non-null  object        
 6   longitud             261362 non-null  object        
 7   municipio            261362 non-null  object        
 8   nombreestacion       261362 non-null  object        
 9   precipitacion_total  261362 non-null  float64       
 10  unidadmedida         261362 non-null  object        
 11  zonahidrografica     261362 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 23.9+ MB


,codigoestacion,codigosensor,date,departamento,descripcionsensor,latitud,longitud,municipio,nombreestacion,precipitacion_total,unidadmedida,zonahidrografica
0,0042077020,0240,2021-07-14,VAUPÉS,Precipitacion,1.26,-70.239,MITÚ,MITU,10.0,mm,VAUPES
1,0012015110,0240,2021-12-12,ANTIOQUIA,Precipitacion,7.671138889,-76.69405556,CHIGORODÓ,CHIGORODO - AUT,0.1,mm,CARIBE - LITORAL


Revisión de datos retirados

In [56]:
merged_df = pd.merge(df_precipitacion, df_precipitacion, how='outer', indicator=True)
removed_data = merged_df[merged_df['_merge'] == 'left_only']
print("Datos Retirados:")
pd.DataFrame(removed_data)
removed_data.info()
removed_data.head(2)

Datos Retirados:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   codigoestacion       0 non-null      object        
 1   codigosensor         0 non-null      object        
 2   date                 0 non-null      datetime64[ns]
 3   departamento         0 non-null      object        
 4   descripcionsensor    0 non-null      object        
 5   latitud              0 non-null      object        
 6   longitud             0 non-null      object        
 7   municipio            0 non-null      object        
 8   nombreestacion       0 non-null      object        
 9   precipitacion_total  0 non-null      float64       
 10  unidadmedida         0 non-null      object        
 11  zonahidrografica     0 non-null      object        
 12  _merge               0 non-null      category      
dtypes: category(1), datetime64[ns](1), 

,codigoestacion,codigosensor,date,departamento,descripcionsensor,latitud,longitud,municipio,nombreestacion,precipitacion_total,unidadmedida,zonahidrografica,_merge


In [57]:
conteo_por_departamento = df_precipitacion['departamento'].value_counts()
conteo_por_departamento = conteo_por_departamento.sort_index()
print(conteo_por_departamento)

departamento
AMAZONAS                                                      762
ANTIOQUIA                                                   18541
ARAUCA                                                       1800
ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA      497
ARCHIPIÉLAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA        76
ATLANTICO                                                    1856
ATLÁNTICO                                                    2876
BOGOTA                                                       2522
BOGOTA D.C.                                                   816
BOGOTÁ                                                         38
BOLIVAR                                                      6613
BOLÍVAR                                                       581
BOYACÁ                                                      28898
CALDAS                                                      14302
CAQUETA                                                      23

In [58]:
df_estandarizado = df_precipitacion.copy()

# Estandarizar el nombre del departamento
df_estandarizado['departamento'] = (
    df_estandarizado['departamento']
    .str.lower()  # Convertir a minúsculas
    .str.replace('d.c.', '', regex=False)  # Retirar "D.C."
    .str.replace('[^\w\s]', '', regex=True)  # Retirar comas, puntos
    .str.replace('[áäâà]', 'a', regex=True)  # Reemplazar tildes
    .str.replace('[éëêè]', 'e', regex=True)
    .str.replace('[ñ]', 'n', regex=True)
    .str.replace('[íïîì]', 'i', regex=True)
    .str.replace('[óöôò]', 'o', regex=True)
    .str.replace('[úüûù]', 'u', regex=True)
    .str.strip()  # Retirar espacios al principio y al final
)
df_estandarizado.loc[df_estandarizado['departamento'].str.contains('san andres providencia', case=False), 'departamento'] = 'san andres providencia'


# Verificar el DataFrame estandarizado
df_estandarizado.head(2)

,codigoestacion,codigosensor,date,departamento,descripcionsensor,latitud,longitud,municipio,nombreestacion,precipitacion_total,unidadmedida,zonahidrografica
0,0042077020,0240,2021-07-14,vaupes,Precipitacion,1.26,-70.239,MITÚ,MITU,10.0,mm,VAUPES
1,0012015110,0240,2021-12-12,antioquia,Precipitacion,7.671138889,-76.69405556,CHIGORODÓ,CHIGORODO - AUT,0.1,mm,CARIBE - LITORAL


In [59]:
conteo_por_departamento = df_estandarizado['departamento'].value_counts()
conteo_por_departamento = conteo_por_departamento.sort_index()
print(conteo_por_departamento)

departamento
amazonas                    762
antioquia                 18541
arauca                     1800
atlantico                  4732
bogota                     3376
bolivar                    7194
boyaca                    28898
caldas                    14302
caqueta                    2404
casanare                   6683
cauca                      8500
cesar                      4315
choco                      6783
cordoba                    9025
cundinamarca              27123
guainia                     633
guaviare                    213
huila                     14489
la guajira                 4053
magdalena                  7136
meta                       6071
narino                    11784
norte de santander        10584
putumayo                   1754
quindio                    5296
risaralda                  6916
san andres providencia     2056
santander                 15752
sucre                      4067
tolima                    13529
valle del cauca           1

In [60]:
import folium
import geopandas as gpd
from folium.plugins import HeatMapWithTime

In [61]:
df_estandarizado['date'] = pd.to_datetime(df_estandarizado['date'])

In [62]:
df_precipitacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261362 entries, 0 to 261361
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   codigoestacion       261362 non-null  object        
 1   codigosensor         261362 non-null  object        
 2   date                 261362 non-null  datetime64[ns]
 3   departamento         261362 non-null  object        
 4   descripcionsensor    261362 non-null  object        
 5   latitud              261362 non-null  object        
 6   longitud             261362 non-null  object        
 7   municipio            261362 non-null  object        
 8   nombreestacion       261362 non-null  object        
 9   precipitacion_total  261362 non-null  float64       
 10  unidadmedida         261362 non-null  object        
 11  zonahidrografica     261362 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 23.9+ MB


In [63]:
df_agrupado = df_estandarizado.groupby(['date', 'departamento']).agg({
    'precipitacion_total': 'sum',
    'unidadmedida': 'first'  # Tomar el primer valor de 'unidadmedida' en cada grupo
}).reset_index()

# Verificar el DataFrame resultante
df_agrupado.info()
df_agrupado.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27885 entries, 0 to 27884
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 27885 non-null  datetime64[ns]
 1   departamento         27885 non-null  object        
 2   precipitacion_total  27885 non-null  float64       
 3   unidadmedida         27885 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 871.5+ KB


,date,departamento,precipitacion_total,unidadmedida
0,2021-01-01,amazonas,49.110,mm
1,2021-01-01,antioquia,227.376,mm
2,2021-01-01,arauca,7.800,mm
3,2021-01-01,atlantico,11.230,mm
4,2021-01-01,bogota,0.110,mm


Se identifica que no hay continuidad en las mediciones de todos los departamentos, encontrando que vaupes, guaviare, guainia y amazonas son los más incompletos.

In [64]:
# Crear un DataFrame con la precipitación total diaria por departamento
df_diario = df_agrupado.groupby(['date', 'departamento']).agg({'precipitacion_total': 'sum'}).reset_index()

# Obtener la lista única de departamentos
lista_departamentos = df_diario['departamento'].unique()

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Diseñar el diseño de la aplicación
app.layout = html.Div([
    # Menú desplegable para seleccionar el departamento
    dcc.Dropdown(
        id='departamento-dropdown',
        options=[{'label': departamento, 'value': departamento} for departamento in lista_departamentos],
        value=lista_departamentos[0],
        multi=False
    ),
    # Gráfico interactivo
    dcc.Graph(id='graph'),
])

# Definir la lógica de interactividad
@app.callback(
    Output('graph', 'figure'),
    [Input('departamento-dropdown', 'value')]
)
def update_graph(selected_departamento):
    # Filtrar el DataFrame por el departamento seleccionado
    df_selected_departamento = df_diario[df_diario['departamento'] == selected_departamento]

    # Pivote del DataFrame para tener departamentos como columnas y fechas como índice
    df_pivot = df_selected_departamento.pivot(index='date', columns='departamento', values='precipitacion_total').reset_index()

    # Agregar una columna 'year' y 'month' al DataFrame pivoteado
    df_pivot['year'] = df_pivot['date'].dt.year
    df_pivot['month'] = df_pivot['date'].dt.month

    # Crear el gráfico con Plotly Express
    fig1 = px.line(df_pivot, x='date', y=selected_departamento, color='year',
                  labels={selected_departamento: 'Precipitación Total'},
                  title=f'Precipitación Total Diaria en {selected_departamento}')

    return fig1

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


In [65]:
# Completar el DataFrame con fechas continuas
df_continuo = df_agrupado.copy()

fechas_continuas = pd.date_range(start=df_continuo['date'].min(), end=df_continuo['date'].max(), freq='D')

df_continuo = pd.DataFrame({'date': fechas_continuas}).drop_duplicates()

# Combinar el DataFrame continuo con el DataFrame original
df_continuo = pd.merge(df_continuo, df_agrupado, on='date', how='left')

# Llenar valores NaN con 0 en la columna 'precipitacion_total'
df_continuo['precipitacion_total'] = df_continuo['precipitacion_total'].fillna(0)

# Pivote del DataFrame
df_pivot_2 = df_continuo.pivot_table(index=['date', 'unidadmedida'], columns='departamento', values='precipitacion_total', aggfunc='sum').reset_index()

# Imprime el DataFrame resultante
df_pivot_2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    959 non-null    datetime64[ns]
 1   unidadmedida            959 non-null    object        
 2   amazonas                555 non-null    float64       
 3   antioquia               940 non-null    float64       
 4   arauca                  854 non-null    float64       
 5   atlantico               936 non-null    float64       
 6   bogota                  894 non-null    float64       
 7   bolivar                 937 non-null    float64       
 8   boyaca                  942 non-null    float64       
 9   caldas                  886 non-null    float64       
 10  caqueta                 889 non-null    float64       
 11  casanare                936 non-null    float64       
 12  cauca                   914 non-null    float64   

In [66]:
from sklearn.impute import SimpleImputer
import pandas as pd

# Supongamos que df_pivot tiene esta estructura:
# df_pivot = ...

df_precipitacion_completo = df_pivot_2.copy()

# Seleccionar solo las columnas numéricas para la imputación
columns_to_impute = df_precipitacion_completo.columns[2:]  # Excluyendo 'date' y 'unidadmedida'

# Crear un imputador SimpleImputer con la estrategia de imputación deseada (por ejemplo, media)
imputer = SimpleImputer(strategy='mean')

# Aplicar la imputación solo a las columnas numéricas
df_precipitacion_completo[columns_to_impute] = imputer.fit_transform(df_precipitacion_completo[columns_to_impute])

# Imprime el DataFrame resultante
df_precipitacion_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    959 non-null    datetime64[ns]
 1   unidadmedida            959 non-null    object        
 2   amazonas                959 non-null    float64       
 3   antioquia               959 non-null    float64       
 4   arauca                  959 non-null    float64       
 5   atlantico               959 non-null    float64       
 6   bogota                  959 non-null    float64       
 7   bolivar                 959 non-null    float64       
 8   boyaca                  959 non-null    float64       
 9   caldas                  959 non-null    float64       
 10  caqueta                 959 non-null    float64       
 11  casanare                959 non-null    float64       
 12  cauca                   959 non-null    float64   

In [67]:
# Obtener la lista de departamentos excluyendo las primeras tres columnas
lista_departamentos = df_precipitacion_completo.columns[2:]

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Diseñar el diseño de la aplicación
app.layout = html.Div([
    # Menú desplegable para seleccionar el departamento
    dcc.Dropdown(
        id='departamento-dropdown',
        options=[{'label': departamento, 'value': departamento} for departamento in lista_departamentos],
        value=lista_departamentos[0],
        multi=False
    ),
    # Gráfico interactivo
    dcc.Graph(id='graph'),
])

# Definir la lógica de interactividad
@app.callback(
    Output('graph', 'figure'),
    [Input('departamento-dropdown', 'value')]
)
def update_graph(selected_departamento):
    # Filtrar el DataFrame por el departamento seleccionado
    df_selected_departamento = df_precipitacion_completo[['date', 'unidadmedida', selected_departamento]].copy()

    # Crear el gráfico con Plotly Express
    fig2 = px.line(df_selected_departamento, x='date', y=selected_departamento,
                  labels={selected_departamento: 'Precipitación Total'},
                  title=f'Precipitación Total Diaria en {selected_departamento}')

    return fig2

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


In [68]:
df_precipitacion_completo.to_csv("processed_tables/precipitacion.csv", index=False)

pendiente terminar mapa de calor

In [69]:
import folium
from folium.plugins import HeatMap

# Crear un mapa centrado en el área de interés (Colombia)
m = folium.Map(location=[4.5709, -74.2973], zoom_start=6)  # Coordenadas aproximadas del centro de Colombia

# Crear una lista de datos para HeatMap
heat_data = [[row['latitud'], row['longitud'], row['antioquia']] for index, row in df_precipitacion_completo.iterrows()]

# Agregar HeatMap al mapa
HeatMap(heat_data).add_to(m)

# Visualizar el mapa en el Jupyter Notebook
m


KeyError: 'latitud'

se identifica que hay una ausencia de data constante en diciembre y enero, algunos departamentos con un vacio mayor, por lo que se tendrá que buscar como completar dicha información.